In [1]:
import pandas as pd
import os

# Directory containing the forecast files
forecasts_dir = '/home/zeal/Documents/6th sem/inv. mangmnt sy/chuimui/forecasts'

# List to store all forecast data
forecasts = []

# Read each file in the forecasts directory
for filename in os.listdir(forecasts_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(forecasts_dir, filename)
        df = pd.read_csv(file_path)
        forecasts.append(df)

# Concatenate all forecast data into a single DataFrame
forecast_data = pd.concat(forecasts, ignore_index=True)


In [2]:
forecast_data.columns

Index(['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'Product', 'Warehouse'], dtype='object')

In [3]:
# Current stock levels data
current_stock = pd.read_csv("/home/zeal/Documents/6th sem/inv. mangmnt sy/chuimui/grouped.csv")
current_stock

,Destination,Category,Quantity
0,Amazon FC - Ahmedabad,BOOKS,3
1,Amazon FC - Ahmedabad,HOME APPLIANCES,50
2,Amazon FC - Amritsar,PET SUPPLIES,32
3,Amazon FC - Bhubaneswar,LAWN AND GARDEN,216
4,Amazon FC - Chandigarh,PRODUCE,35
...,...,...,...
112,Xpressbees Warehouse - Kochi,POULTRY,28
113,Xpressbees Warehouse - Kochi,SCHOOL AND OFFICE SUPPLIES,63
114,Xpressbees Warehouse - Kolkata,CELEBRATION,39
115,Xpressbees Warehouse - Kota,GROCERY II,8


In [4]:
current_stock.rename(columns={'Destination': 'Warehouse'}, inplace=True)
current_stock.columns

Index(['Warehouse', 'Category', 'Quantity'], dtype='object')

In [5]:
current_stock.rename(columns={'Category': 'Product'}, inplace=True)


In [6]:
# Group the forecast data by Product and Warehouse and sum the yhat values
forecast_summary = forecast_data.groupby(['Product', 'Warehouse'])['yhat'].sum().reset_index()

# Merge the forecast summary with the current stock data
merged_data = pd.merge(forecast_summary, current_stock, on=['Product', 'Warehouse'], how='left')

# Fill missing values in CurrentStock with 0
merged_data['Quantity'].fillna(0, inplace=True)


/tmp/ipykernel_116191/367806231.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['Quantity'].fillna(0, inplace=True)


In [7]:
# Calculate the quantity to be restocked or removed
merged_data['QuantityToAdjust'] = merged_data['yhat'] - merged_data['Quantity']

# Define thresholds for labeling
excess_threshold = 1.2  # 20% excess
shortage_threshold = 0.8  # 20% shortage

# Label the stock levels
merged_data['Label'] = merged_data.apply(
    lambda row: 'Excess' if row['Quantity'] > row['yhat'] * excess_threshold else
                'Shortage' if row['Quantity'] < row['yhat'] * shortage_threshold else
                'Good to Go', axis=1
)

# Add a column to display the thresholds
merged_data['Thresholds'] = merged_data.apply(
    lambda row: f"Excess: {row['yhat'] * excess_threshold:.2f}, Shortage: {row['yhat'] * shortage_threshold:.2f}", axis=1
)


# Display the results
print("\nFinal Results:")
print(merged_data[['Product', 'Warehouse', 'yhat', 'QuantityToAdjust', 'Thresholds', 'Label']])


Final Results:
                           Product                          Warehouse  \
0                AUTOMOTIVE_A-0423              Ecom Express - Mysuru   
1                AUTOMOTIVE_B-6128           Blue Dart Hub - Siliguri   
2                AUTOMOTIVE_C-0288               Flipkart FC - Ranchi   
3                 BABY CARE_B-6155  Ecom Express - Thiruvananthapuram   
4                    BEAUTY_E-0838      Delhivery Warehouse - Jodhpur   
5                 BEVERAGES_C-0327      Delhivery Warehouse - Belgaum   
6                     BOOKS_B-6298       Delhivery Warehouse - Nagpur   
7                     BOOKS_X-7078             Amazon FC - Coimbatore   
8                GROCERY II_E-0088               DTDC Hub - Mangalore   
9                 GROCERY I_M-0361                Flipkart FC - Salem   
10                 HARDWARE_Y-002L                   DTDC Hub - Patna   
11       HOME AND KITCHEN I_C-0121                Amazon FC - Udaipur   
12       HOME AND KITCHEN I_S-0359 

In [8]:
insights = merged_data
del merged_data

In [10]:
insights.head()

,Product,Warehouse,yhat,Quantity,QuantityToAdjust,Label,Thresholds
0,AUTOMOTIVE_A-0423,Ecom Express - Mysuru,17953.747343,0.0,17953.747343,Shortage,"Excess: 21544.50, Shortage: 14363.00"
1,AUTOMOTIVE_B-6128,Blue Dart Hub - Siliguri,46101.981415,0.0,46101.981415,Shortage,"Excess: 55322.38, Shortage: 36881.59"
2,AUTOMOTIVE_C-0288,Flipkart FC - Ranchi,85131.762877,0.0,85131.762877,Shortage,"Excess: 102158.12, Shortage: 68105.41"
3,BABY CARE_B-6155,Ecom Express - Thiruvananthapuram,8418.143099,0.0,8418.143099,Shortage,"Excess: 10101.77, Shortage: 6734.51"
4,BEAUTY_E-0838,Delhivery Warehouse - Jodhpur,161785.933761,0.0,161785.933761,Shortage,"Excess: 194143.12, Shortage: 129428.75"


In [12]:
# Drop the QuantityToAdjust column
insights.drop(columns=['Quantity'], inplace=True, errors='ignore')

In [ ]:
insights = insights.merge(curre
nt_stock[['Product', 'Quantity']], on='Product', how='left')

In [15]:
insights.head()

,Product,Warehouse,yhat,QuantityToAdjust,Label,Thresholds,Quantity
0,AUTOMOTIVE_A-0423,Ecom Express - Mysuru,17953.747343,17953.747343,Shortage,"Excess: 21544.50, Shortage: 14363.00",NaN
1,AUTOMOTIVE_B-6128,Blue Dart Hub - Siliguri,46101.981415,46101.981415,Shortage,"Excess: 55322.38, Shortage: 36881.59",NaN
2,AUTOMOTIVE_C-0288,Flipkart FC - Ranchi,85131.762877,85131.762877,Shortage,"Excess: 102158.12, Shortage: 68105.41",NaN
3,BABY CARE_B-6155,Ecom Express - Thiruvananthapuram,8418.143099,8418.143099,Shortage,"Excess: 10101.77, Shortage: 6734.51",NaN
4,BEAUTY_E-0838,Delhivery Warehouse - Jodhpur,161785.933761,161785.933761,Shortage,"Excess: 194143.12, Shortage: 129428.75",NaN
